# Hepstat_Tutorial_Histfactory_Hists
Histfactory example. 

 with following objectives:
 * Create a workspace using histograms
 * Example operations at the workspace level




**Author:** Lailin XU  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Saturday, April 03, 2021 at 05:04 PM.</small></i>

In [1]:
import os

Import the ROOT libraries

In [2]:
import ROOT as R
from math import pow, sqrt
R.gROOT.SetStyle("ATLAS")

Welcome to JupyROOT 6.22/07


Prepare input files
=======================

In [3]:
inputhist = "data/h4l_toy_hists.root"
if not os.path.isfile(inputhist):
  print("Error! No input files found: {}".format(inputhist)) 
  pyhist = "hepstat_tutorial_genhists.py"
  if os.path.isfile(pyhist):
    cmd = "python3 {}".format(pyhist)
    os.system(cmd)

tfin = R.TFile(inputhist, "READ")
data_hist = tfin.Get("obsData")
mass = 125
sig_hist = tfin.Get("sig_{}".format(mass))
bkg_hist = tfin.Get("bkg")
bkg_hup = tfin.Get("bkg_up")
bkg_hdn = tfin.Get("bkg_dn")
sig_hist.Print()
sig_hist.SetDirectory(0)
bkg_hist.SetDirectory(0)
data_hist.SetDirectory(0)
bkg_hup.SetDirectory(0)
bkg_hdn.SetDirectory(0)

TH1.Print Name  = sig_125, Entries= 50, Total sum= 100000


Create a workspace
=======================

Create a Histfactory Measurement
-----------------------

First we set the Parameter of interest, and several constant parameters.

In [4]:
meas = R.RooStats.HistFactory.Measurement("meas", "meas")
meas.SetPOI("mu")


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Set the luminosity constant with a dummy uncertainty of 2%

In [5]:
meas.SetLumi( 1.0 )
meas.SetLumiRelErr( 0.02 )
meas.AddConstantParam("Lumi")

Create a channel and set the measured value of data

In [6]:
chan = R.RooStats.HistFactory.Channel( "SR" )
chan.SetStatErrorConfig(0.05, "Poisson")
chan.SetData( data_hist )

Create the signal sample and set its histogram

In [7]:
signal = R.RooStats.HistFactory.Sample( "signal" )
signal.SetHisto( sig_hist )

Add the parmaeter of interest and a systematic and try to make intelligent choice of upper bound

In [8]:
signal.AddNormFactor( "mu", 1, 0, 3)

Assign a dummy signal normalisation uncertainty (up, down with respect to the nominal)

In [9]:
signal.AddOverallSys( "signal_norm_uncertainty", 0.95, 1.05)

Add the signal sample to the Channel

In [10]:
chan.AddSample( signal )

Create the background sample and set its histogram

In [11]:
background = R.RooStats.HistFactory.Sample( "background" )
background.SetHisto( bkg_hist )

Add the bkg sample to the Channel

In [12]:
chan.AddSample( background )

Add the Channel to the Meas

In [13]:
meas.AddChannel(chan)

Collect the histograms from their files, print some output, 

In [14]:
meas.PrintTree()

Measurement Name: meas	 OutputFilePrefix: 	 POI: mu	 Lumi: 1	 LumiRelErr: 0.02	 BinLow: 0	 BinHigh: 1	 ExportOnly: 0
Constant Params:  Lumi
Channels:
	 Channel Name: SR	 InputFile: 
	 Data:
	 	 InputFile: 	 HistoName: obsData	 HistoPath: 	 HistoAddress: 0x131d176b0
	 statErrorConfig:
	 	 RelErrorThreshold: 0.05	 ConstraintType: Poisson
	 Samples: 
	 	 Name: signal	 	 Channel: SR	 NormalizeByTheory: True	 StatErrorActivate: False
	 	 	 	 	 InputFile: 	 HistName: sig_125	 HistoPath: 	 HistoAddress: 0x131d17e70
	 	 Name: background	 	 Channel: SR	 NormalizeByTheory: True	 StatErrorActivate: False
	 	 	 	 	 InputFile: 	 HistName: bkg	 HistoPath: 	 HistoAddress: 0x131d18260
	 End of Channel SR
[#2] INFO:HistFactory -- End Measurement: meas


Make workspace!
-----------------------

In [15]:
hist2workspace = R.RooStats.HistFactory.HistoToWorkspaceFactoryFast(meas)
ws = hist2workspace.MakeSingleChannelModel(meas, chan)

ws.SetName("myws")
ws.writeToFile("test_hf_ws_{}.root".format(mass))

False

[#2] PROGRESS:HistFactory -- 
-----------------------------------------
	Starting to process 'SR' channel with 1 observables
-----------------------------------------

[#2] INFO:HistFactory -- lumi str = [1,0,10]
[#2] INFO:HistFactory -- lumi Error str = nominalLumi[1,0,1.2],0.02
[#1] INFO:ObjectHandling -- RooWorkspace::import(SR) importing RooStats::HistFactory::FlexibleInterpVar::signal_SR_epsilon
[#2] INFO:HistFactory -- making normFactor: mu
[#2] INFO:HistFactory -- signal_SR has no variation histograms 
[#2] INFO:HistFactory -- processing hist sig_125
[#1] INFO:ObjectHandling -- RooWorkspace::import(SR) importing dataset signal_SRnominalDHist
[#1] INFO:ObjectHandling -- RooWorkspace::import(SR) importing RooHistFunc::signal_SR_nominal
[#1] INFO:ObjectHandling -- RooWorkspace::import(SR) importing RooConstVar::background_SR_epsilon
[#2] INFO:HistFactory -- background_SR has no variation histograms 
[#2] INFO:HistFactory -- processing hist bkg
[#1] INFO:ObjectHandling -- RooWorkspa

Close up

In [16]:
tfin.Close()

Draw all canvases 

In [17]:
from ROOT import gROOT 
gROOT.GetListOfCanvases().Draw()